# 使用 torch.utils.data.Dataset 進行完整的訓練

In [ ]:
# upload Data
!wget -q https://github.com/TA-aiacademy/course_3.0/releases/download/CVCNN_Data/cat_dog.zip
!unzip -q cat_dog

## 匯入套件

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob  # 讀取特定格式路徑
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torchvision.transforms as T

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## 讀取資料

In [ ]:
# 建立一個字典來存放路徑跟標籤資訊
data_dict={'file_name': [], 'type': []}
# 只拿 train 資料中的 .jpg 檔案
for i in glob.glob('cat_dog/train/*.jpg'):
    # i 會類似 cat_dog/train/cat.11996.jpg
    data_dict['file_name'].append(i)
    # 字串處理取出檔案名稱前三個字元來判斷類別
    animal = i.split('/')[-1][:3]
    if animal == 'cat':
        data_dict['type'].append(0)
    elif animal == 'dog':
        data_dict['type'].append(1)
    else:
        print(i)

In [ ]:
# 將字典轉換成 DataFrame
datalist = pd.DataFrame(data_dict)
shuffled_df = datalist.sample(frac=1, random_state=2)  # 打亂順序

In [ ]:
shuffled_df.head()

In [ ]:
len(shuffled_df)

## 切分訓練/驗證集

In [ ]:
# 切分訓練/測試資料
train_data = shuffled_df[:500]
val_data = shuffled_df[500:1000]
test_data = shuffled_df[1000:3000]

## 資料前處理：

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        self.transform = T.Compose([
            T.Resize((256, 256)),
            T.RandomRotation(degrees=15),
            T.ToTensor(),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx, 0]
        img = Image.open(img_path)
        img = self.transform(img)
        label = self.df.iloc[idx, 1]
        return img, label

In [ ]:
# Build dataset
train_dataset = ImageDataset(train_data)
val_dataset = ImageDataset(val_data)
test_dataset = ImageDataset(test_data)

In [ ]:
plt.figure(figsize=(13, 7))
for i in range(8):
    img, label = train_dataset[i]
    plt.subplot(2, 4, i+1)
    plt.imshow(img.permute(1, 2, 0))
    plt.title(f"Label: {label}")
plt.show()

In [ ]:
# build dataloader
BATCH_SIZE = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## 建立神經網路

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, padding='same'),
    nn.ReLU(),
    nn.Conv2d(32, 32, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2), # img_size // 2
    nn.Conv2d(32, 64, 3, padding='same'),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3, padding='same'),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2), # img_size // 4

    nn.Flatten(),
    nn.Linear(64*64*64, 2)
)

print(model)

In [ ]:
def train_epoch(model, optimizer, loss_fn, train_dataloader, val_dataloader):
    # 訓練一輪
    model.train()
    total_train_loss = 0
    total_train_correct = 0
    for x, y in tqdm(train_dataloader, leave=False):
        optimizer.zero_grad() # 梯度歸零
        x, y = x.to(device), y.to(device) # 將資料移至GPU
        y_pred = model(x) # 計算預測值
        loss = loss_fn(y_pred, y) # 計算誤差
        loss.backward() # 反向傳播計算梯度
        optimizer.step() # 更新模型參數
        total_train_loss += loss.item()
        total_train_correct += ((y_pred.argmax(dim=1) == y).sum().item())

    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_acc = total_train_correct / len(train_dataloader.dataset)

    return avg_train_loss, avg_train_acc

def test_epoch(model, loss_fn, val_dataloader):
    # 驗證一輪
    model.eval()
    total_val_loss = 0
    total_val_correct = 0
    with torch.no_grad():
        for x, y in val_dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            total_val_loss += loss.item()
            total_val_correct += ((y_pred.argmax(dim=1) == y).sum().item())

    avg_val_loss = total_val_loss / len(val_dataloader)
    avg_val_acc = total_val_correct / len(val_dataloader.dataset)

    return avg_val_loss, avg_val_acc

def run(epochs, model, optimizer, loss_fn, train_loader, valid_loader):
    train_loss_log = []
    val_loss_log = []
    train_acc_log = []
    val_acc_log = []

    for epoch in tqdm(range(epochs)):
        avg_train_loss, avg_train_acc = train_epoch(model, optimizer, loss_fn, train_loader, valid_loader)
        avg_val_loss, avg_val_acc = test_epoch(model, loss_fn, valid_loader)
        train_loss_log.append(avg_train_loss)
        val_loss_log.append(avg_val_loss)
        train_acc_log.append(avg_train_acc)
        val_acc_log.append(avg_val_acc)
        print(f'Epoch: {epoch}, Train Loss: {avg_train_loss:.3f}, Val Loss: {avg_val_loss:.3f} \
    | Train Acc: {avg_train_acc:.3f}, Val Acc: {avg_val_acc:.3f}')
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log

In [ ]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

## 開始訓練

In [ ]:
logs = run(30, model, optimizer, loss_fn, train_loader, val_loader)

## 測試資料 (模擬沒有答案的測試資料)

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df
        # No random rotation
        self.transform = T.Compose([
            T.Resize((256, 256)),
            T.ToTensor(),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx, 0]
        img = Image.open(img_path)
        img = self.transform(img)
        label = self.df.iloc[idx, 1]
        return img, label

test_dataset = TestDataset(test_data)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# Inference
model.eval()
y_pred_list = []
y_true_list = []

with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        y_pred = model(x)
        y_pred_list.append(y_pred)
        y_true_list.append(y)

y_pred_list = torch.cat(y_pred_list, dim=0)
y_true_list = torch.cat(y_true_list, dim=0)


In [ ]:
print(y_pred_list[:5])

## 解析模型預測結果，並填入Dataframe中

In [ ]:
preditc_label = y_pred_list.argmax(dim=1).cpu().numpy()

In [ ]:
test_df = pd.DataFrame({
    'file_name': test_data['file_name'],
    'prediction': preditc_label
})


In [ ]:
test_df.head()